<a href="https://colab.research.google.com/github/NLP-CISUC/smartedu-aqg/blob/main/Generating_Ranking_Distractors_PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

## MultiEight-04

In [ ]:
#Download data
!wget https://raw.githubusercontent.com/jdportugal/multieight_pt_facts/master/QnAnFacts.txt -O questions.txt

--2023-05-11 23:15:21--  https://raw.githubusercontent.com/jdportugal/multieight_pt_facts/master/QnAnFacts.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74123 (72K) [text/plain]
Saving to: ‘questions.txt’

questions.txt       100%[===================>]  72.39K  --.-KB/s    in 0.01s   

2023-05-11 23:15:21 (6.28 MB/s) - ‘questions.txt’ saved [74123/74123]



In [ ]:
import pandas as pd

ficheiro = "questions.txt"
questions = []

with open(ficheiro) as f:
    lines = f.readlines()
    added = True
    for txt in lines:
      txt = txt.strip()
      if txt.startswith("F:"):
        q = [txt[2:], None, None]
        added = False
      elif txt.startswith("Q:"):
        q[1] = txt[2:]
      elif txt.startswith("A:"):
        q[2] = txt[2:]
        if not added:
          questions.append(q)
          added = True


df = pd.DataFrame(questions, columns=['F', 'Q', 'A'])

docs = df['F'].tolist()
print(docs)

['O prémio Nobel foi atribuido a Thomas Mann em 1929.', 'O Cesare Romiti é o administrador-delegado da Fiat.', 'A prisão de San Vittore encontra-se na cidade de Milão.', 'O hertz é a unidade de medida da frequência.', 'O aumento da população mundial por ano é de 94 milhões.', 'O nome próprio do juiz Borselino é Paolo.', 'Foram mortos três guarda-costas no atentado ao juiz Falcone.', 'Shimon Peres é o o ministro dos negócios estrangeiros israelita.', 'Ariel Sharon ocupa o cargo de líder do Likud.', 'Prémio Nobel da Paz foi atribuído a Yasser Arafat em 1994.', 'Jerusalém é a cidade santa dos judeus.', 'O número de refugiados palestinianos no Líbano eleva-se a 400.000.', 'A Intifada rebentou no ano de 1987.', 'os pilotos suicidas japoneses chamavam-se kamikaze.', 'A França é a principal fornecedora de armas a países do Terceiro Mundo.', 'A bomba atómica sobre Hiroshima foi lançada em 1945', 'Hitler pertencia ao partido nazi.', 'A língua se fala na Alemanha é o alemão.', 'Seis milhões de j

## SQuAD

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.3 MB/s eta 0:00:00


In [ ]:
import json
from datasets import load_dataset, load_metric

files = ['squad-train-v1.1.json', 'squad-dev-v1.1.json']

for file in files:

    # Opening JSON file & returns JSON object as a dictionary
    f = open(file, encoding="utf-8")
    data = json.load(f)

    # Iterating through the json list
    entry_list = list()
    id_list = list()

    for row in data['data']:
        title = row['title']

        for paragraph in row['paragraphs']:
            context = paragraph['context']

            for qa in paragraph['qas']:
                entry = {}

                qa_id = qa['id']
                question = qa['question']
                answers = qa['answers']

                entry['id'] = qa_id
                entry['title'] = title.strip()
                entry['context'] = context.strip()
                entry['question'] = question.strip()

                answer_starts = [answer["answer_start"] for answer in answers]
                answer_texts = [answer["text"].strip() for answer in answers]
                entry['answers'] = answer_texts

                entry_list.append(entry)

    reverse_entry_list = entry_list[::-1]

    # for entries with same id, keep only last one (corrected texts by he group Deep Learning Brasil)
    unique_ids_list = list()
    unique_entry_list = list()
    for entry in reverse_entry_list:
        qa_id = entry['id']
        if qa_id not in unique_ids_list:
            unique_ids_list.append(qa_id)
            unique_entry_list.append(entry)

    # Closing file
    f.close()

    new_dict = {}
    new_dict['data'] = unique_entry_list

    file_name = 'pt_' + str(file)
    with open(file_name, 'w') as json_file:
        json.dump(new_dict, json_file)

In [ ]:
# Defining the dataset
datasets = load_dataset('json',
                        data_files={'train': 'pt_squad-dev-v1.1.json',
                                    'validation': 'pt_squad-train-v1.1.json'},
                        field='data')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-132163de5271774d/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Dataset Details
datasets['validation']

Dataset({
    features: ['answers', 'title', 'id', 'context', 'question'],
    num_rows: 87510
})

In [ ]:
#datasets['train'][1]

def get_squad_sample(n: int):
  question = datasets['validation'][n]['question']
  answer = datasets['validation'][n]['answers'][0]
  context = datasets['validation'][n]['context']

  return {'id': n, 'C': context, 'Q':question, 'A':answer}

target_ids = [71204, 18916, 21295, 64811, 27324, 61949, 3427, 58512, 1412, 48694, 26190, 16299, 34295, 79540, 60449, 75514, 79878, 41738, 83183, 76011, 47079, 46462, 65639, 9148, 65192, 69069, 56035, 53799, 79889, 74331, 21156, 64828, 13285, 82877, 23818, 45696, 54814, 70154, 51522, 27005, 58040, 17537, 6745, 56030, 84990, 16566, 10994, 79472, 33334, 23180, 38965, 70180, 47146, 49662, 60756, 55528, 55553, 77950]

entries = []
for id in target_ids:
  entry = get_squad_sample(id)
  entries.append(entry)

import pandas as pd
df = pd.DataFrame(entries)
df.to_excel("target_questions.xlsx")
#print(df)


# Distractors

## BERTopic

In [ ]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 63.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 95.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.1 MB/s eta 0:00:00
  Cr

In [ ]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer, util
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
import numpy as np


# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("rufimelo/bert-large-portuguese-cased-sts") #for Portuguese!

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# Step 4 - Tokenize topics
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
pt_stop_words = stopwords.words('portuguese')
vectorizer_model = CountVectorizer(stop_words=pt_stop_words)

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# All steps together
topic_model = BERTopic(
  embedding_model=embedding_model,    # Step 1 - Extract embeddings
  umap_model=umap_model,              # Step 2 - Reduce dimensionality
  hdbscan_model=hdbscan_model,        # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,  # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,          # Step 5 - Extract topic words
  #diversity=0.5,                      # Diversify topic words
  calculate_probabilities=True,
  nr_topics=10,
  verbose=True
)

topics, probs = topic_model.fit_transform(docs)
topic_model.get_topic_info()

KeyboardInterrupt: ignored

In [ ]:
import random

df_topics = pd.DataFrame({'topic': topics, 'document': docs})
n_topics = len(pd.unique(df_topics['topic']))

for n in range(-1, n_topics-1):
  docs_topic = list(df_topics[df_topics.topic == n].document)
  print('topic_'+str(n)+':', len(docs_topic))
  random.shuffle(docs_topic)
  concat_doc = ''.join([dd+' ' for dd in docs_topic[:3]])
  print(concat_doc)
  for d in docs_topic[:3]:
    e = df[df.F == d]
  #  print('\t', e['F'])
    print('\tQ:', e['Q'].values[0])
    print('\tA:', e['A'].values[0])


## WordNet

In [ ]:
#WordNet distractors -> co-hyponyms

import string

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
import logging

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
ps = PorterStemmer()

logger = logging.getLogger('my_logger')
logging.basicConfig()

LANGUAGE = 'portuguese'
LANG_WN = 'por'

#TODO: incluir antónimos?
def getWordNetDistractors(answer, question):
    distractors = []
    try:
      prepAnswer, tokensAnswer = preprocessText(answer, LANGUAGE)
    except Exception as e:
      logger.error("Exception in function preprocessText: %s", e)

    # we try to get potential distractors looking for cohyponyms
    # i.e. words that share the same hyperonym
    cohyponyms = []
    try:
      cohyponyms = getCohyponyms(prepAnswer, LANG_WN)
    except Exception as e:
      logger.error("Exception in function getCohyponyms: %s", e)
    for cohyponym in cohyponyms:
      if cohyponym != answer and cohyponym != prepAnswer and cohyponym not in distractors and cohyponym not in question:
        distractors.append(cohyponym)

    # if number of distractors selected is not sufficient, repeat for processed answer
    #if len(distractors) == 0:
    #    try:
    #        cohyponyms = getCohyponyms(phrase, LANG_WN)
    #    except Exception as e:
    #        logger.error("Exception in function getCohyponyms in getDistractors (distractorsWordNet.py): %s", e)
    #    for cohyponym in cohyponyms:
    #        if cohyponym != answer and cohyponym != phrase and cohyponym not in distractors:
    #            distractors.append(cohyponym)

    # if number of distractors selected is still not sufficient, repeat for answer's tokens
    if len(distractors) == 0 and ' ' in answer:
      for ta in tokensAnswer:

        try:
          cohyponymsToken = getCohyponyms(ta, LANG_WN)
        except Exception as e:
          logger.error("Exception in function getCohyponyms: %s", e)

        for co in cohyponymsToken:
          if ' ' not in co and co not in question: # single token -> no spaces!
            distractor = answer.replace(ta, co)
            if distractor != answer and distractor != prepAnswer and distractor not in distractors:
              distractors.append(distractor)

    return distractors


# lower case, remove punctuation and stopwords
def preprocessText(text, language):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token not in stopwords.words(language)]
    return text, tokens


# cohyponyms share the same hyperonym
# first we look for hyperonyms of the word, then proceed to get the hyponyms of those hyperonyms
def getCohyponyms(word, language):
    word = word.replace(" ","_")

    cohyponyms = []
    synonyms = wordnet.synsets(word, lang=language)

    for synonym in synonyms:
        hypernyms = synonym.hypernyms()
        for hypernym in hypernyms:
            hyponyms = hypernym.hyponyms()
            for hyponym in hyponyms:
                if hyponym not in synonyms:
                    lemmas = hyponym.lemmas(lang=language)
                    for lemma in lemmas:
                        name = lemma.name()
                        name = name.replace("_"," ")
                        cohyponyms.append(name)

    return cohyponyms


getWordNetDistractors('álcool', 'Qual é a substância mais comumente abusada durante a adolescência nos EUA?')
getWordNetDistractors('boina preta', 'Que peça do uniforme foi substituída pelo boné de patrulha?')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


['balackava preta',
 'balaclava preta',
 'Barrete preta',
 'barrete preta',
 'fez preta',
 'tarbush preta',
 'quepe preta',
 'gorro preta']

## Numbers

In [ ]:
import re

def getNumberDistractors(answer, question, window=10):
  distractors = []
  #if re.match('[0-9]+', answer):
  # Check if answer is number then get range based on answer
  if answer.isnumeric():
    num = int(answer)
    distractors.extend(number_range(num, window))

  # Check if number in answer then get range based on vaue
  if len(distractors) == 0:
    tokens = nltk.word_tokenize(answer)
    for tok in tokens:
      if tok.isnumeric():
        nums = number_range(int(tok), window)
        for n in nums:
          distractors.append(answer.replace(tok, str(n)))

  return distractors

# Get range based on number value and window
def number_range(num, window):
  nums = []
  for n in range(num-window, num+window):
    if n != num and str(n) not in question:
      nums.append(str(n))
  return nums


#getNumberDistractors('1917', 'Em que ano aconteceu o acontecimento?')
getNumberDistractors('Entre 1914 e 1917', 'Em que ano aconteceu o acontecimento?')

['Entre 1904 e 1917',
 'Entre 1905 e 1917',
 'Entre 1906 e 1917',
 'Entre 1907 e 1917',
 'Entre 1908 e 1917',
 'Entre 1909 e 1917',
 'Entre 1910 e 1917',
 'Entre 1911 e 1917',
 'Entre 1912 e 1917',
 'Entre 1913 e 1917',
 'Entre 1915 e 1917',
 'Entre 1916 e 1917',
 'Entre 1917 e 1917',
 'Entre 1918 e 1917',
 'Entre 1919 e 1917',
 'Entre 1920 e 1917',
 'Entre 1921 e 1917',
 'Entre 1922 e 1917',
 'Entre 1923 e 1917',
 'Entre 1914 e 1907',
 'Entre 1914 e 1908',
 'Entre 1914 e 1909',
 'Entre 1914 e 1910',
 'Entre 1914 e 1911',
 'Entre 1914 e 1912',
 'Entre 1914 e 1913',
 'Entre 1914 e 1914',
 'Entre 1914 e 1915',
 'Entre 1914 e 1916',
 'Entre 1914 e 1918',
 'Entre 1914 e 1919',
 'Entre 1914 e 1920',
 'Entre 1914 e 1921',
 'Entre 1914 e 1922',
 'Entre 1914 e 1923',
 'Entre 1914 e 1924',
 'Entre 1914 e 1925',
 'Entre 1914 e 1926']

## spacy (Context)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install spacy
#!python -m spacy download pt_core_news_md
!python -m spacy download pt_core_news_lg

import spacy
#nlp = spacy.load('en_core_web_md')
nlp = spacy.load('pt_core_news_lg')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-11 23:19:03.700100: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 23:19:04.775425: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-11 23:19:06.166879: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-11 23:19:06.167487: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node rea

In [ ]:
# NE distractors: replicate for some POS, e.g. V and ADJ?

import nltk
import string
from nltk.corpus import stopwords
from nltk.corpus import wordnet
#from nltk.stem import PorterStemmer
#ps = PorterStemmer()

import logging
logger = logging.getLogger('my_logger')
logging.basicConfig()

def getContextDistractors(answer, question, text):
  doc = nlp(text)

  distractors = getNEDistractors(answer, text, question, doc)
  if not distractors:
    distractors = getPOSDistractors(answer, text, question, doc)

  return list(set(distractors))


def getNEDistractors(answer, text, question, doc):
  hasNE = None
  otherNEs = []
  for ent in doc.ents:
    if ent.text in answer:
      hasNE = (ent.text, ent.label_)
    else:
      otherNEs.append((ent.text, ent.label_))

  distractors = []
  if hasNE:
    for ne in otherNEs:
      if ne[1] == hasNE[1] and ne[0] not in question:
        d = answer.replace(hasNE[0], ne[0])
        if d not in answer:
          distractors.append(d)

  return distractors


def getPOSDistractors(answer, text, question, doc, pos_apply=['VERB', 'ADJ']):
  targets = []
  others = []
  for tok in doc:
    #print('*', tok, tok.pos, tok.tag_, pos_apply)
    if tok.tag_ in pos_apply or (not pos_apply and not tok.is_stop):

      if tok.text in answer:
        targets.append((tok.text, tok.tag_))
      else:
        others.append((tok.text, tok.tag_))


  distractors = []
  for targ in targets:
    for tok in others:
      if tok[1] == targ[1] and tok[0] not in question:
        distractors.append(answer.replace(targ[0], tok[0]))

  return distractors

## spacy (word2vec)

In [ ]:
import spacy
import numpy as np
nlp = spacy.load('pt_core_news_lg')

import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('omw-1.4')
LANG_WN = 'por'


def most_similar(word, topn=5):
  ms = nlp.vocab.vectors.most_similar(
      nlp(word).vector.reshape(1,nlp(word).vector.shape[0]), n=topn+1)
  words = [nlp.vocab.strings[w] for w in ms[0][0]]
  words.remove(word)
  #distances = ms[2]
  return words

#print(most_similar("rato")[0])
#print(most_similar("ano")[0])
#print(most_similar("renda")[0])
#print(most_similar("cantar")[0])

def get_synonyms(word, language):
  return wordnet.synonyms(word, lang=language)

def get_hypernyms(word, language):
  synsets = wordnet.synsets(word, lang=language)
  hyper_sets = set()
  for ss in synsets:
    hyper_sets.update(ss.hypernyms())
  hypernyms = set()
  for hs in hyper_sets:
    for lemma in hs.lemmas(lang=language):
      hypernyms.add(lemma.name().replace('_', ' '))

  return list(hypernyms)


def getSimilarDistractors(answer, question, pos_apply=['NOUN', 'ADJ']):
  doc = nlp(answer)

  replace = []
  for tok in doc:
    if tok.tag_ in pos_apply and not tok.is_stop and not tok.is_oov:
      similar = most_similar(tok.text)
      if similar:
        synonyms = get_synonyms(tok.text, language=LANG_WN)
        hypernyms = get_hypernyms(tok.text, language=LANG_WN)
        for s in similar:
          if s.lower() not in synonyms and s.lower() not in hypernyms: #similar, but not a synonym
            replace.append((tok.text, s))


  distractors = []
  for r in replace:
    if r[1].lower() not in answer.lower() and r[1] not in question:
        distractors.append(answer.replace(r[0], r[1]))

  return distractors


#def removeSynonyms(word, similar, language):
#  synonyms = wordnet.synonyms(word, lang=language)
#  return [s for s in similar if s not in synonyms]


#getSimilarDistractors('vingar o gato do irmão que vive em espanha', 'qual a razão da viagem?')

getSimilarDistractors('boina preta', 'Que peça do uniforme foi substituída pelo boné de patrulha?')

#hypernyms('cão', language='por')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


['jaqueta preta',
 'gorro preta',
 'blusa preta',
 'camisola preta',
 'boina branca',
 'boina vermelha',
 'boina prateada',
 'boina marrom',
 'boina roxa']

## DBPedia

In [ ]:
!pip install SPARQLWrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.8 MB/s eta 0:00:00


In [ ]:
# Distractors DBPedia

import spacy
from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper('https://dbpedia.org/sparql')
import string
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

#import distractorsNER
#from nltk.stem import PorterStemmer
#ps = PorterStemmer()

import logging
logger = logging.getLogger('my_logger')
logging.basicConfig()

LANGUAGE = 'portuguese'
LANG_LABEL = 'pt'

# main function
def getDBPediaDistractors(answer, question):
  try:
    processedAnswer, tokensAnswer = preprocessAnswer(answer, LANGUAGE)
  except Exception as e:
    logger.error("Exception in function preprocessDistractor in getDistractors (distractorsDBpedia.py): %s", e)

    # first letter upper case and lower case to maximize number of distractors discovered
    # if len(answer)>1:
    #    possibleAnswers = [answer[0].lower() + answer[1:], answer[0].upper() + answer[1:]]
    #else:
    #    possibleAnswers = [answer]

  possibleAnswers = [processedAnswer, processedAnswer.lower()]

  uris = []
  for pa in possibleAnswers:
    uri = searchURIs(pa, LANG_LABEL)
    if len(uri) > 0:
      uris.extend(uri)

  distractors = []
  try:
    distractors = selectDistractors(uris, question, answer, LANG_LABEL)
  except Exception as e:
    logger.error("Exception in function selectDistractors in getDistractors (distractorsDBpedia.py): %s", e)

    # if number of distractors selected is not sufficient, repeat for processed answer
    # if len(distractors) == 0 and len(processedAnswer) > 1:
    #     possibleAnswers = [processedAnswer[0].lower() + processedAnswer[1:], processedAnswer[0].upper() + processedAnswer[1:]]
    #     try:
    #         distractors = selectDistractors(possibleAnswers, LANG_LABEL)
    #     except Exception as e:
    #         logger.error("Exception in function selectDistractors in getDistractors (distractorsDBpedia.py): %s", e)

  # if number of distractors selected still not sufficient, repeat answer's tokens
  if len(distractors) == 0 and ' ' in answer:
    #print('TOKS:', tokensAnswer)
    for tok in tokensAnswer:
      uris = searchURIs(tok, LANG_LABEL)

      if len(uris) > 0:
        #print('URIS:', uris)
        candidates = []
        try:
          candidates = selectDistractors(uris, question, answer, LANG_LABEL)
        except Exception as e:
          logger.error("Exception in function selectDistractors in getDistractors (distractorsDBpedia.py): %s", e)

        for c in candidates:
          if ' ' not in c: # single token -> no spaces!
            distractor = answer.replace(tok, c)
            if distractor != answer:
              distractors.append(distractor)

  return list(set(distractors))


# Lower case, remove punctuation and stopwords
def preprocessAnswer(answer, language):
    #answer = answer.lower()
    answer = answer.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(answer)
    tokens = [token for token in tokens if token not in stopwords.words(language)]
    #answer = " ".join(tokens)
    return answer, tokens


# CohyponymsBroaderNarrower was the only function that seemed to have interesting results
def selectDistractors(uris, question, answer, language):
    distractors = []

    try:
      distractorsAux = relatedConcepts(uris, language)
      if distractorsAux:
        distractors.extend(distractorsAux)
    except Exception as e:
      logger.error("Exception in function relatedConcepts in selectDistractors (distractorsDBpedia.py): %s", e)

    return [d for d in set(distractors) if d not in answer and d not in question]


# search for DBpedia URIs that have as label the given string in the english language
def searchURIs(value, language):
  uris = []

  query = '''
      SELECT ?uri
      WHERE {
        ?uri rdfs:label "'''+value+'''"@'''+language+''' .
      }
      '''

  try:
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    for binding in qres['results']['bindings']:
      uris.append(binding['uri']['value'])
  except:
    pass

  return uris

# Search for concepts of the same category / subject of the answer.
def relatedConcepts(uris, language):
  distractors = set()

  for uri in uris:
    try:
      concepts = skosBroaderNarrower(uri, language)
      if concepts:
        distractors.update(concepts)
    except Exception as e:
      logger.error("Exception in function skosBroaderNarrower in relatedConcepts: %s", e)

    #print(uri, 'SKOS', distractors)

  if len(distractors) == 0:
    for uri in uris:
      try:
        concepts = conceptsSameSubject(uri, language)
        if concepts:
          distractors.update(concepts)
      except Exception as e:
        logger.error("Exception in function conceptsSameSubject in relatedConcepts: %s", e)

      #print(uri, 'SUBJECT', distractors)

  return list(distractors)


# search for concepts of the same category
def skosBroaderNarrower(value, language):
    URIs = []

    query = '''
        SELECT ?label
        WHERE {
            <'''+value+'''> skos:broader ?value . ?co skos:broader ?value . ?co rdfs:label ?label . FILTER langMatches(lang(?label),"'''+language+'''")
        }
        '''

    # print('SPARQL', query)
    try:
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        qres = sparql.query().convert()
        for binding in qres['results']['bindings']:
            URIs.append(binding['label']['value'])
    except:
        pass

    return URIs

# search for concepts of the same category
def conceptsSameSubject(value, language):
    URIs = []

    query = '''
        PREFIX dct:<http://purl.org/dc/terms/>
        SELECT ?label
        WHERE {
            <'''+value+'''> dct:subject ?value . ?co dct:subject ?value . ?co rdfs:label ?label . FILTER langMatches(lang(?label),"'''+language+'''")
        }
        '''

    print('SPARQL', query)
    try:
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        qres = sparql.query().convert()
        for binding in qres['results']['bindings']:
            URIs.append(binding['label']['value'])
    except:
        pass

    return URIs



#getDBPediaDistractors('Jean MacDonald', 'Qual cantor se apresentou no funeral de Bell?')
getDBPediaDistractors('Portas USB', 'O que normalmente é desligado sempre que o computador está desligado?')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


SPARQL 
        PREFIX dct:<http://purl.org/dc/terms/>
        SELECT ?label
        WHERE { 
            <http://dbpedia.org/resource/USB> dct:subject ?value . ?co dct:subject ?value . ?co rdfs:label ?label . FILTER langMatches(lang(?label),"pt")
        }
        


['Portas Benzimidazol',
 'Portas FireWire',
 'Portas 2,4-Dinitrofenol',
 'Portas Airbag',
 'Portas 1-Propanotiol',
 'Portas C-MET',
 'Portas Glutaraldeído',
 'Portas Chinelo',
 'Portas Desoxiguanosina',
 'Portas Skee-Ball',
 'Portas 1-Butanol',
 'Portas Harmina',
 'Portas Nitazoxanida',
 'Portas Bazedoxifeno',
 'Portas Hexanoil-CoA',
 'Portas Pentaclorofenol',
 'Portas N-acetilglicosamina',
 'Portas Xenônio',
 'Portas Fudepen',
 'Portas Sorbitol',
 'Portas Internet',
 'Portas EDTA',
 'Portas Catana',
 'Portas Machete',
 'Portas Flurazepam',
 'Portas Calculadora',
 'Portas Scrabble',
 'Portas Colesterol',
 'Portas Isopreno',
 'Portas Diclofenaco',
 'Portas Salicilamida',
 'Portas Gabapentina',
 'Portas Levotiroxina',
 'Portas Torquímetro',
 'Portas Microfone',
 'Portas Ansacrina',
 'Portas Fax',
 'Portas Olaparib',
 'Portas Pentoxifilina',
 'Portas Hidroximetilfurfural',
 'Portas 2-Naftol',
 'Portas Doxepina',
 'Portas Telefone',
 'Portas Fortran',
 'Portas Vitascópio',
 'Portas Tiamina

## Transformers

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#T5:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-race-Distractor")
model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-race-Distractor")
maxLength = 1024

def getT5Distractors(answer, question, context):
  input_text = " ".join([question, tokenizer.sep_token, answer, tokenizer.sep_token, context])
  inputs = tokenizer(input_text, return_tensors="pt")
  outputs = model.generate(**inputs, max_new_tokens=128)
  distractors = tokenizer.decode(outputs[0], skip_special_tokens=False)
  distractors = distractors.replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")
  distractors = [y.strip() for y in distractors.split(tokenizer.sep_token)]

  return distractors

# Test

## Test

In [ ]:
text = '''
        Joe Biden nasceu em Scranton, a 20 de novembro de 1942.
        Ele é um advogado e político norte-americano que serve atualmente como o 46.º presidente dos Estados Unidos.
        Filiado ao Partido Democrata, serviu também como o 47.º vice-presidente de 2009 a 2017 no governo Obama.
        Entre 1973 e 2009, exerceu seis mandatos consecutivos como senador pelo Delaware, período em que presidiu importantes comités do Senado.
        '''

text_en = '''
        Joe Biden was born in Scranton, on November 20, 1942.
        He is an American lawyer and politician currently serving as the 46th president of the United States.
        He is a member of the Democrat Party and also served as the 47th Vice-President, from 2009 to 2017 in the Obama administration.
        Between 1973 and 2009, he served for six consecutive terms as a Delaware senator, while chairing important comittees in the Senate.
'''

#distractors = getContextDistractors('Estados Unidos', text)
#distractors = getNEDistractors('foi em Delaware', text)
#print(distractors)
#getNEDistractors('em 1942', text, 3)


#getWordNetDistractors('arma', 3)

#distractors = getBARTDistractors(text_en, "What is Joe Biden's party?", 'Democratic', 10)
#distractors = getT5Distractors(text_en, "What state did Joe Biden served as a senator for?", 'Delaware', 10)
#distractors = getT5Distractors(text, "Qual é o partido de Joe Biden?", 'Democrata', 10)
#distractors = getT5Distractors(text, "Por que estado Joe Biden foi senador?", 'Delaware', 10)

#print(distractors)


'''
import random

df_topics = pd.DataFrame({'topic': topics, 'document': docs})
n_topics = len(pd.unique(df_topics['topic']))
top_n_concat = 3

for n in range(-1, n_topics-1):
  docs_topic = list(df_topics[df_topics.topic == n].document)
  print('topic_'+str(n)+':', len(docs_topic))
  random.shuffle(docs_topic)
  concat_doc = ''.join([dd+' ' for dd in docs_topic[:top_n_concat]])
  print(concat_doc)
  for d in docs_topic[:top_n_concat]:
    e = df[df.F == d]
    print('\tQ:', e['Q'].values[0])
    print('\tA:', e['A'].values[0])

    distractors = getContextDistractors(e['A'].values[0], e['Q'].values[0], concat_doc)
    print('\tContext:', distractors)

    distractors = getWordNetDistractors(e['A'].values[0], e['Q'].values[0])
    print('\tWordNet:', distractors)

    distractors = getDBPediaDistractors(e['A'].values[0], e['Q'].values[0])
    print('\tDBPedia:', distractors)

    distractors = getSimilarDistractors(e['A'].values[0], e['Q'].values[0])
    print('\tSimilar:', distractors)
'''

import random
#distractors = getDBPediaDistractors('Reino de Portugal e dos Algarves')
distractors = getDBPediaDistractors(answer='Cagliari, Gênova, Livorno, Nápoles, Tunis', question='Para quais cidades as balsas de Palermo viajam?')
random.shuffle(distractors)
print(distractors)

SPARQL 
        PREFIX dct:<http://purl.org/dc/terms/>
        SELECT ?label
        WHERE { 
            <http://dbpedia.org/resource/Cagliari> dct:subject ?value . ?co dct:subject ?value . ?co rdfs:label ?label . FILTER langMatches(lang(?label),"pt")
        }
        
SPARQL 
        PREFIX dct:<http://purl.org/dc/terms/>
        SELECT ?label
        WHERE { 
            <http://dbpedia.org/resource/Livorno> dct:subject ?value . ?co dct:subject ?value . ?co rdfs:label ?label . FILTER langMatches(lang(?label),"pt")
        }
        
SPARQL 
        PREFIX dct:<http://purl.org/dc/terms/>
        SELECT ?label
        WHERE { 
            <http://dbpedia.org/resource/Naples> dct:subject ?value . ?co dct:subject ?value . ?co rdfs:label ?label . FILTER langMatches(lang(?label),"pt")
        }
        
['Cagliari, Gênova, Livorno, Felitto, Tunis', 'Cagliari, Gênova, Livorno, Bamburgo, Tunis', 'Cagliari, Gênova, Livorno, Perquenco, Tunis', 'Sedilo, Gênova, Livorno, Nápoles, Tunis', 'Ospe

## Test SQuAD

In [ ]:
# Test with SQuAD
import random
from string import punctuation

def get_squad_sample(n: int):
  question = datasets['validation'][n]['question']
  answer = datasets['validation'][n]['answers'][0]
  context = datasets['validation'][n]['context']

  return {'Context': context, 'Q':question, 'A':answer}

n_rand = 25
rand_inds = [random.randint(0, len(datasets['validation'])) for n in range(n_rand)]

# Tests based on samples
for i in rand_inds:
  print(f'---------- Index: {i} ----------')
  sample_1 = get_squad_sample(i)

  if sample_1['A'].count(' ') > 2:
    continue


  sample_1['A'] = sample_1['A'].strip(punctuation).strip()

  # Question and Answer
  print(f"Text: {sample_1['Context']}")
  print(f"Q: {sample_1['Q']}\nA: {sample_1['A']}\n")

  # Distractors
  distractors = getContextDistractors(sample_1['A'], sample_1['Q'], sample_1['Context'])
  print('Context:', distractors)

  distractors = getNumberDistractors(sample_1['A'], sample_1['Q'])
  print('Numbers:', distractors)

  distractors = getWordNetDistractors(sample_1['A'], sample_1['Q'])
  print('WordNet:', distractors)

  distractors = getDBPediaDistractors(sample_1['A'], sample_1['Q'])
  print('DBPedia:', distractors)

  distractors = getSimilarDistractors(sample_1['A'], sample_1['Q'])
  print('Similar:', distractors)

  print('\n')

---------- Index: 20681 ----------
Text: O santo padroeiro de Palermo é Santa Rosalia, que é amplamente reverenciada. Em 14 de julho, as pessoas de Palermo comemoram o Festino anual, o evento religioso mais importante do ano. O Festino é uma procissão que atravessa a rua principal de Palermo para comemorar o milagre atribuído a Santa Rosalia que, acredita-se, libertou a cidade da Peste Negra em 1624. Seus restos foram descobertos em uma caverna no Monte Pellegrino, e seus restos foram transportados pela cidade três vezes, banindo a praga. Há um santuário marcando o local onde seus restos mortais foram encontrados e podem ser alcançados através de uma viagem de ônibus panorâmico da cidade.
Q: Quem disse que terminou a Peste Negra em 1624?
A: Santa Rosalia

Context: ['Monte Pellegrino', 'Palermo']
Numbers: []
WordNet: []
SPARQL 
        PREFIX dct:<http://purl.org/dc/terms/>
        SELECT ?label
        WHERE { 
            <http://dbpedia.org/resource/Rosalia> dct:subject ?value . ?co 

# Ranking Options

In [ ]:
#FitBERT

!pip install fitbert

from fitbert import FitBert
from transformers import BertForMaskedLM, BertTokenizer, AutoTokenizer, AutoModelForMaskedLM, GPT2Tokenizer, GPT2LMHeadModel

BERT_BASE_PT = 'neuralmind/bert-base-portuguese-cased'
BERT_LARGE_PT = 'neuralmind/bert-large-portuguese-cased'
GPT_PT = 'pierreguillou/gpt2-small-portuguese'

tokenizer_bert_b = BertTokenizer.from_pretrained(BERT_BASE_PT)
model_bert_b = BertForMaskedLM.from_pretrained(BERT_BASE_PT)
fb_b = FitBert(model=model_bert_b, tokenizer=tokenizer_bert_b)

tokenizer_bert_l = BertTokenizer.from_pretrained(BERT_LARGE_PT)
model_bert_l = BertForMaskedLM.from_pretrained(BERT_LARGE_PT)
fb_l = FitBert(model=model_bert_l, tokenizer=tokenizer_bert_l)

model_gpt = GPT2LMHeadModel.from_pretrained(GPT_PT)
tokenizer_gpt = GPT2Tokenizer.from_pretrained(GPT_PT)


#FitBERT
def sentence_rank_fitbert(fb, sentence, options):
  return fb.rank(sentence+' ***mask***', options=options)

#GPT-2
def sentence_rank_gpt(model, tokenizer, sentence, options):

  sentences = [sentence+' '+opt for opt in options]
  sents_score = []
  for sent in sentences:
    tokens_tensor = tokenizer.encode(sent, add_special_tokens=False, return_tensors="pt")
    #predictions
    loss = model(tokens_tensor, labels=tokens_tensor)[0]
    sents_score.append((sent, np.exp(loss.cpu().detach().numpy())))

  sents_score.sort(key=lambda x: x[1], reverse=True)
  options_rank = [options[sentences.index(ss[0])] for ss in sents_score]
  return options_rank

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.7/81.7 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fitbert: filename=fitbert-0.10.0-py3-none-any.whl size=218456 sha256=5aadc53a69f3056a5420d341dab21c4df28faea0dd5cec1d2b58902c728f33d8
  Stored in directory: /root/.cache/pip/wheels/ac/a7/18/f1ee25cdca155f58a344ad42813d47d62b9e8823dd90a01f86
  Created wheel for dill: filename=dill-0.2.5-py3-none-any.whl size=52435 sha256=62d904e31d98912cb1ecf662572ff14e9cf0098c1db5caf10e93e288cf8b3b9a
  Stored in directory: /root/.cache/pip/wheels/25/c6/b5/9ece3456a958311d04047d31e61bb53235e4713a5e3f3e182a
Successfully built fitbert dill
  Attempting uninstall: dill
  

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


device: cuda
using custom model: ['BertForMaskedLM']


Some weights of the model checkpoint at neuralmind/bert-large-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


device: cuda
using custom model: ['BertForMaskedLM']


In [ ]:
MAX_POR_TIPO = 10

for n in range(-1, n_topics-1):
  docs_topic = list(df_topics[df_topics.topic == n].document)
  print('topic_'+str(n)+':', len(docs_topic))
  random.shuffle(docs_topic)
  concat_doc = ''.join([dd+' ' for dd in docs_topic[:top_n_concat]])
  print(concat_doc)
  for d in docs_topic[:top_n_concat]:
    e = df[df.F == d]
    print('\tQ:', e['Q'].values[0])
    print('\tA:', e['A'].values[0])

    dc = getContextDistractors(e['A'].values[0], e['Q'].values[0], concat_doc)
    dc = sample(dc, MAX_POR_TIPO)
    dw = getWordNetDistractors(e['A'].values[0], e['Q'].values[0])
    dw = sample(dw, MAX_POR_TIPO)
    dd = getDBPediaDistractors(e['A'].values[0], e['Q'].values[0])
    dd = sample(dd, MAX_POR_TIPO)

    distractors = dc + dw + dd

    rank_bert = sentence_rank_fitbert(fb, e['Q'].values[0], options=distractors)
    print(rank_bert[:4])

NameError: ignored

# Evaluation

In [ ]:
#Gerar uma estrutura que tenha várias entradas: contexto + pergunta + resposta + (distrator, tipo, rank (de acordo com vários métodos))
#Sugestão: duas tabelas
#uma com id + contexto + pergunta + resposta [pode ser criada primeiro]
#outra com id + distrator + tipo + ranks... [idealmente criada com base na anterior]

import random
from string import punctuation
import pandas as pd


def get_squad_sample(n: int):
  question = datasets['validation'][n]['question']
  answer = datasets['validation'][n]['answers'][0]
  context = datasets['validation'][n]['context']

  return {'id': n, 'C': context, 'Q':question, 'A':answer}


def distractor_entries(id, distractors, type_name, max, shuffle=True):
  entries = []
  if shuffle:
    random.shuffle(distractors)
  distractors = distractors[:min(len(distractors), max)]
  for i, d in enumerate(distractors):
    #entries.append((i, d, type_name, i))
    entries.append({'id': id, 'distractor': d, 'type': type_name, 'rank': i})
  return entries



#all_indexes = np.arange(len(datasets['validation']))
#available_indexes = np.setdiff1d(all_indexes, hugo_ids)
#rand_inds = np.random.choice(available_indexes, size=N_QUESTIONS, replace=False)
#rand_inds = [index.item() for index in rand_inds]

rand_inds = [
73243, 2927, 66157, 62467, 2303, 85404, 1381, 18044, 73904, 60591, 34755, 35704, 23598, 63308, 44429,
22751, 83859, 37015, 62538, 7008, 36101, 72597, 50434, 80363, 27286, 16549, 6263, 7778, 78633, 3780,
57448, 69535, 14064, 69409, 14095, 29099, 77213, 62564, 50761, 71168, 30957, 60559, 27549, 1290, 16954,
32847, 7518, 24296, 44455, 14423, 50280, 71064, 35059, 69267, 54023, 49171, 40776, 21046, 53941, 10439,
86021, 23028, 48531, 52482, 77130, 61719, 47020, 84672, 73550, 81548, 62276, 1207, 10867, 35868, 41720,
74415, 39240, 7695, 56475, 74154, 31658, 34786, 17325, 74346, 64320, 8300, 4359, 78850, 5262, 30339]

print(len(rand_inds))


# Folha perguntas
entries = []
for id in rand_inds:
  entry = get_squad_sample(id)
  entries.append(entry)
df_questions = pd.DataFrame(entries)
df_questions.to_excel("questions.xlsx")


MAX_DISTS = 3
sel_distractors = []
for i in rand_inds:


  print(f'---------- Index: {i} ----------')
  idx = rand_inds.index(i)
  print('...', idx, 'de', len(rand_inds))
  sample_1 = get_squad_sample(i)

  if sample_1['A'].count(' ') > 2:
    continue

  #fazer ao carregar o dataset?
  sample_1['A'] = sample_1['A'].strip(punctuation).strip()
  print(f"T: {sample_1['C']}")
  print(f"Q: {sample_1['Q']}\nA: {sample_1['A']}\n")

  context = getContextDistractors(sample_1['A'], sample_1['Q'], sample_1['C'])
  entries = distractor_entries(i, context, 'context', MAX_DISTS)
  sel_distractors.extend(entries)
  #df_distractors.append(entries)
  #pd.concat([df_distractors, entries])
  #print('Context:', entries)

  numbers = getNumberDistractors(sample_1['A'], sample_1['Q'])
  entries = distractor_entries(i, numbers, 'numbers', MAX_DISTS)
  sel_distractors.extend(entries)
  #df_distractors.append(entries)
  #pd.concat([df_distractors, entries])
  #print('Numbers:', entries)

  wn = getWordNetDistractors(sample_1['A'], sample_1['Q'])
  entries = distractor_entries(i, wn, 'wordnet', MAX_DISTS)
  sel_distractors.extend(entries)
  #df_distractors.append(entries)
  #pd.concat([df_distractors, entries])
  #print('WordNet:', entries)

  dbpedia = getDBPediaDistractors(sample_1['A'], sample_1['Q'])
  entries = distractor_entries(i, dbpedia, 'dbpedia', MAX_DISTS)
  sel_distractors.extend(entries)
  #df_distractors.append(entries)
  #pd.concat([df_distractors, entries])
  #print('DBPedia:', entries)

  sim = getSimilarDistractors(sample_1['A'], sample_1['Q'])
  entries = distractor_entries(i, sim, 'sim', MAX_DISTS)
  sel_distractors.extend(entries)
  #df_distractors.append(entries)
  #pd.concat([df_distractors, entries])
  #print('Similar:', entries)

  #distractors = context + numbers + wn + dbpedia + sim
  max_for_all = MAX_DISTS*10
  all_distractors = context[:min(len(context), max_for_all)] + numbers[:min(len(numbers), max_for_all)] + wn[:min(len(wn), max_for_all)] + dbpedia[:min(len(dbpedia), max_for_all)] + sim[:min(len(sim), max_for_all)]
  #so' para teste...
  #random.shuffle(distractors)
  #distractors = distractors if len(distractors) < 15 else distractors[:15]
  #print(len(distractors))

  rank_rand = distractor_entries(i, all_distractors, 'rand', MAX_DISTS, shuffle=True)

  rank_bert_b = sentence_rank_fitbert(fb_b, sample_1['Q'], options=all_distractors)
  #print('FITBERT', rank_bert[:MAX_POR_TIPO])
  entries = distractor_entries(i, rank_bert_b, 'bert_b', MAX_DISTS, shuffle=False)
  sel_distractors.extend(entries)
  #for j, d in enumerate(rank_bert):
  #  df_tmp = df[(df['id'] == str(i)) & (df['distractor'] == d)]
  #  if len(df_tmp) == 0:
  #    df.loc[len(df)] = {'id': i, 'distractor': d, 'type': 'bert', 'rank_bert': j}
  #  else:
  #    df.loc[df['id'] == i & df['distractor'] == d, 'rank_bert'] = j

  rank_bert_l = sentence_rank_fitbert(fb_l, sample_1['Q'], options=all_distractors)
  #print('FITBERT', rank_bert[:MAX_POR_TIPO])
  entries = distractor_entries(i, rank_bert_l, 'bert_l', MAX_DISTS, shuffle=False)
  sel_distractors.extend(entries)

  rank_gpt = sentence_rank_gpt(model_gpt, tokenizer_gpt, sample_1['Q'], options=all_distractors)
  #print('FITBERT', rank_bert[:MAX_POR_TIPO])
  entries = distractor_entries(i, rank_gpt, 'gpt2', MAX_DISTS, shuffle=False)
  sel_distractors.extend(entries)

  print('\n')

print(len(sel_distractors), sel_distractors)
df_distractors = pd.DataFrame(sel_distractors)
#print(df_distractors)
df_distractors.to_excel("distractors.xlsx")


# Folha classificação
entries = []
for id in rand_inds:
  row = get_squad_sample(id)
  dd_rows = df_distractors[df_distractors["id"] == id]
  dd = list(set(dd_rows['distractor']))
  random.shuffle(dd)
  for d in dd:
    entries.append({'id': id, 'distractor': d})

df_class = pd.DataFrame(entries)
df_class.to_excel("class.xlsx")

90
---------- Index: 73243 ----------
... 0 de 90
T: A autoridade executiva em Santa Helena é exercida pela rainha Elizabeth II e é exercida em seu nome pelo governador de Santa Helena. O governador é nomeado pela rainha a conselho do governo britânico. A Defesa e os Negócios Estrangeiros continuam sendo de responsabilidade do Reino Unido.
Q: Quem é responsável pela Defesa e Relações Exteriores?
A: o Reino Unido

SPARQL 
        PREFIX dct:<http://purl.org/dc/terms/>
        SELECT ?label
        WHERE { 
            <http://dbpedia.org/resource/Reino> dct:subject ?value . ?co dct:subject ?value . ?co rdfs:label ?label . FILTER langMatches(lang(?label),"pt")
        }
        


---------- Index: 2927 ----------
... 1 de 90
T: Os Irmãos Plymouth são um movimento evangélico conservador, de igreja baixa, cuja história pode ser rastreada até Dublin, Irlanda, no final da década de 1820, originária do anglicanismo. Entre outras crenças, o grupo enfatiza a sola scriptura. Os irmãos geralment

In [ ]:
# Folha classificação
entries = []
for id in rand_inds:
  row = get_squad_sample(id)
  dd_rows = df_distractors[df_distractors["id"] == id]
  dd = list(set(dd_rows['distractor']))
  random.shuffle(dd)
  for d in dd:
    entries.append({'id': id, 'distractor': d})

df_class = pd.DataFrame(entries)
df_class.to_excel("class.xlsx")

# Cross data

In [ ]:
import pandas as pd
import random

perguntas = pd.read_excel(open('output_distratores.xlsx', 'rb'), sheet_name='perguntas')
#print(perguntas)
distratores = pd.read_excel(open('output_distratores.xlsx', 'rb'), sheet_name='distratores')
#print(distratores)

for index, row in perguntas.iterrows():
  print(str(row['id'])+"\t"+row['C']+'\t'+row['Q']+'\t'+str(row['A']))
  dd_rows = distratores[distratores["id"] == row['id']]
  dd = list(set(dd_rows['distractor']))
  random.shuffle(dd)
  for d in dd:
    print('\t\t\t' + d)